In [16]:
from openai import AzureOpenAI
from openai.types import Embedding
from dotenv import load_dotenv
import pytest
import os
import logging
import tiktoken 
from json import JSONDecodeError
from sklearn.metrics.pairwise import cosine_similarity
from cluestar import plot_text

load_dotenv('.env.shared')
load_dotenv('.env.secret')

6ccabd95d76e4f5084e6447e5f0be172


In [116]:
openai_client = AzureOpenAI(
    api_key=test,
    base_url=os.getenv('OPENAI_API_BASE'),
    api_version=os.getenv('OPENAI_API_VERSION')
)


In [91]:
system_prompt = """The user will provide you with a paragraph of text and a metadata string.
The data will follow this format:

METADATA: 
PARAGRAPH:

The paragraphs and metadata can be in different languages. Always answer in english. They all come from the same domain of oil and gas exploration and production. 

You will create questions that can be answered from the text, alongside the two answers for each question.
The first answer should be as short and explicit as possible. Put extra weight on making it as short as possible with no filler words at all. Use the exact words and phrasing from the text.
The second answer should be answered using natural language.
The metadata should always be explicitly written in the questions and explicitly written in the second answer.
You may also use exact sentences or shorten the sentences to phrases to generate examples.
You may paraphrase the content in sentences or change them slightly but you must preserve both language and style of individual texts.
If there are no examples to generate then create an empty list. 
Provide a confidence score between 0 and 1 for each of the question answer pairs where 1 is full confidence in the answer, and 0 is no confidence in the answer.

You are to produce a JSON formatted output, with JSON output only in the following form:

[{"Q": "<Question 1a>", "A1": "<Answer 1b>", "A2": "<Answer 1c>", "C" : "<Confidence 1d>"}, 
 {"Q": "<Sentence 2a>", "A1": "<Sentence 2b>", "A2": "<Answer 2c>", "C" : "<Confidence 2d>"},
 ...,
 {"Q": "<Sentence Na>", "A1": "<Sentence Nb>", "A2": "<Answer Nc>", "C" : "<Confidence Nd>"}]

You are to determine yourself how many examples N you can come up with, but make as many as possible from the text provided, preferably at least 10. It is very important to make sure its always JSON formatted.
"""

In [88]:
user_prompt = lambda a, b: """
METADATA: """+a+""" 

PARAGRAPH: """+b 

data_list = [
{
"metadata" : "15/5-1",
"paragraph" : 'IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.'
},
{
"metadata" :  '30/7-7',
"paragraph" : 'The clay calculation from MN plot is excluded because this gives zero percent clay in some interval and this does not fit to any of the other clay indicatores. These clay intervals became hydrocarbon bearing with MN as clay indicator. These intervals Below 4975 the sand become more shaly. From the Gamma ray the clean lin (GR min) seems to be between 15 and 41 API. If the 41 API is used the gamma ray as clay indicator will dominate in this interval, because of this 15 API used like the zone above. 2.3 HYDROCARBON DENSITY The hydrocarbon density is set ot even if no or almost no hydrocarbon effect can be seen from the logs. The probability of having heavy oil at this high pressure and temperature is very low. With following effect no high hydrocarbon effect on the -logs could be expected: High water saturation, hydrocarbon fluid of volatile oil or condensate and possibly deep invation. 2.4 CUT orF VALUE Cut off porosity 10% Cut off clay 40% Cut off Sw 70% The Sw cut off value uf 70% is due to the fact that most probable the hydrocarbon fluid is gas or gas condensate. If hydrocarbon is oil the cut off values should be lower. With heavy oil the possibility of very little movable oil is present. By using these cut offs high frequency of all three cut offs criteria acting at same intervals are observed. This tendency might confirm the validity of the cut offs.'
},
{"metadata" : '25/4-1',
"paragraph" : '''FORMATION TESTER S.P.E. PL 28 Is DAIE 19/10/72 OPE RA10 St art Ended R P00 DURAND '014. . 6E1 total 11. of 01 Company Witness CARRF 26/14-1 WELL 'FORMATION JIJRASSIQUC GENERAL INFORMATIONS MUD (Drilling Mud) Density Viscosity 50 InfiPPEKA completion letIdSoi in, 39 Source of it trate Hole diameter Jurassic R eleIVOir Reason for test ing FOR MAftObi evpi Casing  Level  SAMPLINd Late wiper trip Carts' tonnidgy Sandstone Granulornetry  Porosity ______________17 II Permeability .dims Salinity of interstitial water Static Reservoir Pre ssure Head of mud RwP-fl 1flnc .04 of test Testing depth 3504,6 absolute depth -34 81 OVERALL RESULTS r.Ace C.ASCII.DIF TOOLS Combo FIT 1 ool FIT or FIT _______ Mode of placement CM. Diameter of char ges, 1/8 kind of Setting I Pe or pad HARD Sompiing chamber 110 or ./2 21 3/4 Hydraulic circuit reguiat Or TEST Pressure /COI IL Pad Weisser& psi a shut in Hydrostatic pr...... reeding prinstire cifrapolatied si Pr lessor./ ft owing Start 7200 End. /600 Flowing ti rne Sampling chamber till up rot, ___________ 70 % BHP. reading m2 extrapolated Sampling chamber pressure reading at surrace .4acto. Permeability based on pressure calcoJations ________________________ from build up pressure curve 0  10000 _1_ GA RESULTS 75.2 cf 4. gasoline 500 CC Is' It WATER. FILTRATE- FILIRAIE 2000 CC Rent 24 16 'IC DRY EVENTUAL FAILURE kg m2 kg/cm2 _ duration 1_111S1 Observation5.____________ FYI tranafe. CHsemenIs ELF NORGE'''
}
]
# metadata = '34/10-2'
# paragraph = 'Litologi Sedimentbeskrivelse basert pa daglig telex fra Statfjord. Dyp tekst refererer til KB. De Overste ca 500 metrene av brOnnen bestar av sand og kan trolig henfOres til Pleistocene. Fra 5-600 til ca 900 bestar sedimentene hovedsakelig av leire og leirstein med sandlag. Tuffrik skifer er telex rapportert fra 915 dyp, men mangefargete leirsteiner: (varicoloured claystones) er be- skrevet fra 700 m, slik at tuffsonen muligens ligger grunnere enn ca 900 m. intervallet 790 - 850 kan det ut fra GR/Sonic logg muligens antydes en tuffsone, men fra dette intervallet er tuff ikke rapportert. Balder er derfor antydet innenfor intervallet 790 - ca 000 m. Fra ca 000 til 878 bestar sedimentene av gra leirstein med lag av gra og brunlig kalksten og enkelte sandstenslag. Ut fra loggene virker hele denne sekvensen noksa monoton, og den antas representere Shetland gruppen. Fra 878 til 885 viser GR-loggen en klar hot-shale, mens sedimentene dette intervallet er beskrevet som en rOd leirsten ned til 940 der det er et loggebrudd. Sekven- sen fra topp hot shale til loggebruddet pa 940 anses for & tilhOre Humber gruppen Intervallet fra 940 til 125 med interbedded kull, sandsten, siltsten og skifer representerer Brent-Formasjonen. En markert overgang pa GR-log pa 125 til en relativt homogen skifersekvens markerer overgangen til Dunlin Forma- sjonen. Den er beskrevet som sandsten og siltsten Ovre del av formasjonen med gradvis overgang til en homogen sek- yens med mOrk gra skifer. Overgangen til den mer sandige Statfjord-Formasjonen viser ved tydelige utslag pa GR og Sonic logger pa ca 300 m. Statfjord-Formasjonen bestar av sandsten og gra skifer interbedded. Bunnen av Statfjord er noe usikker, men er her satt til dyp 540 som SLA/JaV-notat am 34/10-1 og 2.'

metadata = '15/5-1'
paragraph = 'A piled steel jacket of this sort is probably not the most cost effective fixed structure for this location. Section 1.1.5 deals with the alternatives that exist and from this superficial examination, it is probable that the single leg jack-up concept, with pre-commissioned topside equipment installed over predrilled template would be comparable in cost. a. Influence of Method of Installation Barge launching the structure on location is the conventional method of installation, followed by controlled flooding to up- right the structure and final sinking to the seabed. If such a method is adopted, at least one face of the structure must have two parallel lauch tracks, and strong launch truss must be incorporated internally. This implies that at least legs must be used arranged in two rows of 4. The major disadvantage of such an eight-leg structure is that the wave force attracted are larger than for more open four legged structure . This is only true, however, if provision for installation is not considered and for heavier structures. The conventional North Sea approach has been to build large self-floating legs, which attract considerable extra wave force, (e.g. Thistle, Ninian, Magnus) or fabricate purpose built floa- taion rafts, which can be submerged with the structure atta- ched (e.g. Froties). The Forties system is not now favoured because of the lack of resuability of the floatation rafts and in each case, considerable extra problems of fabrication are introduced. The most complex and time consuming fabrication for self-floating jackets is for the floatation legs on the ground,,and this must be substantially complete before the main jacket structure can be built. This.complicatps and extends the fabrication process. A four legged structure without provision for installation will generally weight less because of the reduced wave forces and more open structure. It is not as redundant as an eight legged'


prompt = user_prompt(metadata, paragraph)

In [87]:
# for dict in data_dict:
#     print(user_prompt(dict["metadata"], dict["paragraph"]))


METADATA: 15/5-1 

PARAGRAPH: IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.

METADATA: 30/7-7 

PARAGRAPH: The clay calculation from MN plot is excluded because this gives zero percent clay in some interval and this does not fit to any of the other clay indicatores. 

In [80]:
# Load the shared environment variables, not secrets
load_dotenv("../.env.shared")
load_dotenv("../.env.secret")


openai_client = AzureOpenAI(
    api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"]
)




completion = openai_client.chat.completions.create(model=os.environ["GPT432k_DEPLOYMENT"], 
                                            messages=[{"role": "system", 
                                                       "content": system_prompt}, 
                                                       {"role": "user", "content": prompt}])

In [81]:
# print(completion.choices[0].message.content)

[{"Q": "What does section 1.1.5 of the 15/5-1 metadata deal with?", "A1": "Alternatives that exist for the cost effective fixed structure for this location.", "A2": "Section 1.1.5 of the metadata 15/5-1, addresses the alternatives that are there which could be optimal and cost effective for the fixed structure in this location.", "C" : "1"}, 

 {"Q": "What concept does the 15/5-1 metadata propose as comparable in cost to a piled steel jacket?", "A1": "Single leg jack-up concept, with pre-commissioned topside equipment installed over predrilled template.", "A2": "According to the metadata 15/5-1, the single leg jack-up idea wherein pre-commissioned topside equipment is installed over the predrilled template is proposed to be similar in cost to a piled steel jacket.", "C" : "0.9"},

 {"Q": "What is the conventional method of installation as per the paragraph on 15/5-1 metadata?", "A1": "Barge launching the structure on location, followed by controlled flooding to up- right the structure 

In [94]:
from concurrent.futures import ThreadPoolExecutor

user_prompt = lambda a, b: """
METADATA: """+a+""" 

PARAGRAPH: """+b 

def ask_question(data):
    prompt = user_prompt(data["metadata"], data["paragraph"])
    # system_prompt = data["metadata"]
    response = openai_client.chat.completions.create(
        model=os.environ["GPT432k_DEPLOYMENT"], 
        messages=[
            {"role": "system", "content": system_prompt}, 
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def main():
    with ThreadPoolExecutor() as executor:
        responses = list(executor.map(ask_question, data_list))
    return responses

responses = main()


In [108]:
import pandas as pd
import ast
df = pd.DataFrame()
counter = 0
for list_dict in responses:
    try:
        print(ast.literal_eval(list_dict))
    except:
        print("Not proper output")
    # df1 = pd.DataFrame(list)
    # counter +=1
    # df = df.concat([df1], ignore_index=True)

    # print(counter)
    
    # print(list)

[{'Q': 'What investigation was conducted in well 15/5-1?', 'A1': 'Investigation of the logs', 'A2': 'An investigation of the logs was conducted in well 15/5-1.', 'C': '1'}, {'Q': 'What was discovered in well 15/5-1 due to the investigation of logs?', 'A1': 'hydrocarbon sand in the interval 3558-3614 RKBx', 'A2': 'The investigation of the logs in well 15/5-1 revealed the presence of hydrocarbon sand in the interval 3558-3614 RKBx.', 'C': '1'}, {'Q': 'What is the total thickness of the discovered interval in well 15/5-1?', 'A1': '56 m', 'A2': 'The total thickness of the discovered interval in Well 15/5-1 is 56 meters.', 'C': '1'}, {'Q': 'In the context of well 15/5-1, how is the sand section distributed?', 'A1': 'subdivided into four separate pay zones', 'A2': 'In well 15/5-1, the sand section is subdivided into four separate pay zones.', 'C': '1'}, {'Q': 'What separates each pay zone in the well 15/5-1 study?', 'A1': 'thin impermeable layers', 'A2': 'In the well 15/5-1 study, each pay z

In [113]:
responses_edited = ['text here [{"Q": "What investigation was conducted in well 15/5-1?", "A1": "Investigation of the logs", "A2": "An investigation of the logs was conducted in well 15/5-1.", "C" : "1"},\n {"Q": "What was discovered in well 15/5-1 due to the investigation of logs?", "A1": "hydrocarbon sand in the interval 3558-3614 RKBx", "A2": "The investigation of the logs in well 15/5-1 revealed the presence of hydrocarbon sand in the interval 3558-3614 RKBx.", "C" : "1"},\n {"Q": "What is the total thickness of the discovered interval in well 15/5-1?", "A1": "56 m", "A2": "The total thickness of the discovered interval in Well 15/5-1 is 56 meters.", "C" : "1"},\n {"Q": "In the context of well 15/5-1, how is the sand section distributed?", "A1": "subdivided into four separate pay zones", "A2": "In well 15/5-1, the sand section is subdivided into four separate pay zones.", "C" : "1"},\n {"Q": "What separates each pay zone in the well 15/5-1 study?", "A1": "thin impermeable layers", "A2": "In the well 15/5-1 study, each pay zone is separated by thin impermeable layers.", "C" : "1"},\n {"Q": "In the 15/5-1 study, how much of the oil column is equivalent to the upper zone?", "A1": "83%", "A2": "In the 15/5-1 study, the upper zone is equivalent to 83% of the oil column.", "C" : "1"},\n {"Q": "What was used as depth reference in the log evaluation of well 15/5-1?", "A1": "The CP1 log", "A2": "In the log evaluation of well 15/5-1, the CP1 log was used as a depth reference.", "C" : "1"},\n {"Q": "What is the RKB elevation in well 15/5-1?", "A1": ",.2-5m", "A2": "The RKB elevation in well 15/5-1 is ,.2-5m.", "C" : "0.7"},\n {"Q": "What was used in the cut-off criteria of the log evaluation for well 15/5-1?", "A1": "The CP1 log", "A2": "The CP1 log was used in the cut-off criteria of the log evaluation for well 15/5-1.", "C" : "1"},\n {"Q": "What is one thing they did not find in well 15/5-1?", "A1": "No hydrocarbon water contact was identified", "A2": "In well 15/5-1, they did not identify any hydrocarbon water contact.", "C" : "1"}]', '[{"Q": "What is one of the results of using MN as a clay indicator in paragraph 30/7-7?", "A1": "Clay intervals became hydrocarbon bearing", "A2": "In the metadata paragraph 30/7-7, when MN is used as a clay indicator, it resulted in the clay intervals becoming hydrocarbon bearing", "C" : "1"},\n {"Q": "What change is reported in paragraph 30/7-7 for the intervals below 4975?", "A1": "The sand becomes more shaly", "A2": "According to the metadata paragraph 30/7-7, there is a change in the interval below 4975 with the sand becoming more shaly", "C" : "1"},\n {"Q": "What are the values indicated for clean lin (GR min) in the Gamma ray in paragraph 30/7-7?", "A1": "Between 15 and 41 API", "A2": "In paragraph 30/7-7, the values for clean lin (GR min) in the Gamma ray are reported to be between 15 and 41 API", "C" : "1"},\n {"Q": "In paragraph 30/7-7, what is the hydrocarbon density set to?", "A1": "Even", "A2": "In paragraph 30/7-7, the hydrocarbon density is set to be \'even\' or consistent.", "C" : "1"},\n {"Q": "Which high pressure and temperature characteristic is stated in paragraph 30/7-7?", "A1": "Probability of having heavy oil is very low", "A2": "The metadata paragraph 30/7-7 states that the characteristic of having heavy oil at high pressure and temperature is very low", "C" : "1"},\n {"Q": "What are some expected characteristics related to high water saturation in paragraph 30/7-7?", "A1": "No high hydrocarbon effect on the logs", "A2": "Regarding high water saturation, paragraph 30/7-7 expresses that there would be no high hydrocarbon effect on the logs", "C" : "1"},\n {"Q": "What does the cut-off porosity value indicate in paragraph 30/7-7?", "A1": "10%", "A2": "In paragraph 30/7-7, a cut-off porosity value of 10% is mentioned", "C" : "1"},\n {"Q": "What are the cut-off values for clay and Sw as per paragraph 30/7-7?", "A1": "Clay 40%, Sw 70%", "A2": "Paragraph 30/7-7 specifies cut-off values of 40% for clay and 70% for Sw", "C" : "1"},\n {"Q": "Why is the Sw cut-off value of 70% used in paragraph 30/7-7?", "A1": "Most probable the hydrocarbon fluid is gas or gas condensate", "A2": "Paragraph 30/7-7 mentions that the Sw cut-off value of 70% is used because it is most probable the hydrocarbon fluid is either gas or gas condensate", "C" : "1"},\n {"Q": "What effects are observed by using these cut-offs in paragraph 30/7-7?", "A1": "High frequency of all three cut offs criteria acting at same intervals", "A2": "Paragraph 30/7-7 notes that with the use of these cut-off values, a high frequency of all three cut-offs criteria acting at the same intervals is observed, suggesting their validity", "C" : "1"}]', '[{"Q": "From the METADATA 25/4-1, what is the reason for testing?", "A1": "FOR MAftObi evpi", "A2": "The reason for the 25/4-1 testing is FOR MAftObi evpi", "C" : "0.4"},\n {"Q": "According to the METADATA 25/4-1, what is the test depth?", "A1": "3504,6 absolute depth", "A2": "The testing depth according to the 25/4-1 data is 3504,6 absolute depth", "C" : "0.7"},\n {"Q": "In the context of METADATA 25/4-1, what kind of setting was used?", "A1": "Pe or pad HARD", "A2": "Pe or pad HARD was the type of setting used in the 25/4-1 context.", "C" : "0.6"},\n {"Q": "From METADATA 25/4-1, what is the salinity of the interstitial water?", "A1": ".dims", "A2": "The salinity of the interstitial water from the 25/4-1 data is .dims", "C" : "0.4"},\n {"Q": "In the METADATA 25/4-1, what is observed about the sampling chamber?", "A1": "110 or ./2 21 3/4", "A2": "In the 25/4-1 data, it is observed that the sampling chamber is marked as 110 or ./2 21 3/4", "C" : "0.6"},\n {"Q": "What was the porosity according to the METADATA 25/4-1?", "A1": "17 II", "A2": "According to the 25/4-1 metadata, the porosity was recorded as 17 II", "C" : "0.7"},\n {"Q": "According to the METADATA 25/4-1, what type of mud was used?", "A1": "InfiPPEKA", "A2": "The type of mud used according to the 25/4-1 data was InfiPPEKA", "C" : "0.6"},\n {"Q": "In the METADATA 25/4-1, what was the diameter of charges?", "A1": "1/8", "A2": "The diameter of the charges in the 25/4-1 metadata was 1/8", "C" : "0.8"},\n {"Q": "From the METADATA 25/4-1, what was the reservoir?", "A1": "Jurassic R eleIVOir", "A2": "According to the 25/4-1 data, the reservoir was marked as Jurassic R eleIVOir", "C" : "0.5"},\n {"Q": "What was the Filtrate according to METADATA 25/4-1?", "A1": "FILIRAIE 2000 CC", "A2": "According to the 25/4-1 metadata, Filtrate was marked as FILIRAIE 2000 CC", "C" : "0.7"}]']
df = pd.DataFrame()
error_list = []
for list_dict in responses_edited:
    try:
        reformed = ast.literal_eval(list_dict)
        df1 = pd.DataFrame(reformed)
        df = pd.concat([df, df1])
    except:
        print("Not proper output")
        error_list.append(list_dict)

df.to_csv("QA_completed", sep=',', index=False, encoding='utf-8')

Not proper output


In [114]:
print(df)

                                                   Q  \
0  What is one of the results of using MN as a cl...   
1  What change is reported in paragraph 30/7-7 fo...   
2  What are the values indicated for clean lin (G...   
3  In paragraph 30/7-7, what is the hydrocarbon d...   
4  Which high pressure and temperature characteri...   
5  What are some expected characteristics related...   
6  What does the cut-off porosity value indicate ...   
7  What are the cut-off values for clay and Sw as...   
8  Why is the Sw cut-off value of 70% used in par...   
9  What effects are observed by using these cut-o...   
0  From the METADATA 25/4-1, what is the reason f...   
1  According to the METADATA 25/4-1, what is the ...   
2  In the context of METADATA 25/4-1, what kind o...   
3  From METADATA 25/4-1, what is the salinity of ...   
4  In the METADATA 25/4-1, what is observed about...   
5  What was the porosity according to the METADAT...   
6  According to the METADATA 25/4-1, what type o